<a href="https://colab.research.google.com/github/alitoast/Exam_IR/blob/fetcher_parser-dev/disperazione.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Librerie

In [1]:
!pip install aiohttp
!pip install asyncio
!pip install rfc3986
!pip install simhash

In [2]:
import pandas as pd
import re
from urllib.parse import urlparse
import urllib.robotparser #per gestire il file robot.txt
import xml.etree.ElementTree as ET  #per gestire i file xml
import requests #per http
import time #per gestire il tempo
from bs4 import BeautifulSoup #per gestire il parsing del html (si può usare anche per xml)
import rfc3986 # per la normalizzazione degli urls
import logging
import asyncio
import aiohttp
import aiofiles
import json
import os
from collections import Counter
import numpy as np
import nltk
import simhash
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus.reader.wordnet import NOUN, VERB, ADJ, ADV
from collections import defaultdict


In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#Fetcher

In [ ]:
#url di prova
# ha sitemap-index ma non priority etc.
start_url_uno = "https://www.dragopublisher.com/it/"
# non ha sitemap
start_url_due = "https://www.wildraccoon.it/shop/"
# sito con sitemap
start_url = "https://www.bbc.com/"

In [4]:
# questo funziona
class UserAgentPolicy:
    def __init__(self, name, header):
        self.name = name
        self.header = header
        self.base_url = None
        self.path_disallow = None
        self.crawl_delay = None
        self.request_rate = None
        self.last_access = 1e9
        self.visited = set()
        self.lock = asyncio.Lock()


class Fetcher:
    def __init__(self, session, useragent_name="Googlebot"):
        self.session = session
        self.useragents = {
            "Googlebot": UserAgentPolicy("Googlebot", 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'),
            "Bingbot": UserAgentPolicy("Bingbot", 'Mozilla/5.0 (compatible; Bingbot/2.0; +http://www.bing.com/bingbot.htm)'),
            "*": UserAgentPolicy("*", 'Mozilla/5.0 (compatible; MyBot/1.0; +http://example.com/bot)'),
        }
        self.default_agent = self.useragents.get(useragent_name, self.useragents["Googlebot"])


    async def check_robots(self, url, useragent=None):
        if not useragent:
            useragent = self.default_agent
        useragent.base_url = url
        new_url = url.rstrip("/") + "/robots.txt"

        rfl = urllib.robotparser.RobotFileParser()
        try:
            async with self.session.get(new_url, timeout=10) as response:
                if response.status == 200:
                    robots_txt = await response.text()
                    rfl.parse(robots_txt.splitlines())
                    logging.info(f"Fetched robots.txt: {new_url}")
                else:
                    logging.warning(f"Failed to fetch robots.txt: HTTP {response.status}")
                    return None
        except aiohttp.ClientError as e:
            logging.error(f"Error fetching robots.txt: {e}")
            return None

        useragent.path_disallow = rfl.parse("Disallow")
        useragent.crawl_delay = rfl.crawl_delay(useragent.name)
        useragent.request_rate = rfl.request_rate(useragent.name)

        return rfl.site_maps()

    async def check_time(self, useragent):
        async with useragent.lock:
            if useragent.request_rate and useragent.crawl_delay:
                request_delay = (useragent.request_rate[1] * 1e9) / useragent.request_rate[0]
                crawl_delay = useragent.crawl_delay * 1e9
                delay = max(request_delay, crawl_delay)
            elif useragent.crawl_delay:
                delay = useragent.crawl_delay * 1e9
            elif useragent.request_rate:
                delay = (useragent.request_rate[1] * 1e9) / useragent.request_rate[0]
            else:
                delay = 1.5 * 1e9  # default delay
            logging.info(f"Delay: {delay / 1e9} seconds")
            now = time.monotonic_ns()
            wait = max(0, (useragent.last_access + delay) - now)
            if wait > 0:
                await asyncio.sleep(wait / 1e9)
            useragent.last_access = time.monotonic_ns()
            logging.info(f"{useragent.name} Last access: {useragent.last_access / 1e9} seconds")

    async def fetch(self, url, useragent=None):
      if not useragent:
          useragent = self.default_agent

      if url in useragent.visited:
          logging.warning(f"[{useragent.name}] Page already visited: {url}")
          return None

      await self.check_time(useragent)

      headers = {'User-Agent': useragent.header}

      start_time = time.perf_counter()
      try:
          async with self.session.get(url, headers=headers, timeout=aiohttp.ClientTimeout(total=20)) as response:
              duration = time.perf_counter() - start_time

              if response.status == 200:
                  content_type = response.headers.get('Content-Type', '').lower()
                  if 'text/html' in content_type:
                      html = await response.text()
                      logging.info(f"[{useragent.name}] Fetched HTML page: {url} in {duration:.2f}s")
                      useragent.visited.add(url)
                      return (html, str(response.url), response.status)
                  else:
                      logging.warning(f"[{useragent.name}] Skipping non-HTML content: {url} (Content-Type: {content_type})")
                      return None
              else:
                  logging.warning(f"[{useragent.name}] Failed to fetch {url}: HTTP {response.status} in {duration:.2f}s")
                  return None

      except asyncio.TimeoutError:
          duration = time.perf_counter() - start_time
          logging.error(f"[{useragent.name}] Timeout after {duration:.2f}s fetching {url}")
          return None
      except aiohttp.ClientError as e:
          duration = time.perf_counter() - start_time
          logging.error(f"[{useragent.name}] Client error fetching {url} after {duration:.2f}s: {e}")
          return None
      except Exception as e:
          duration = time.perf_counter() - start_time
          logging.error(f"[{useragent.name}] Unexpected error fetching {url} after {duration:.2f}s: {e}")
          return None


In [ ]:
# Useragents
useragent_dict = {
    "Googlebot": UserAgentPolicy("Googlebot",'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'),

    "Bingbot": UserAgentPolicy("Bingbot",'Mozilla/5.0 (compatible; Bingbot/2.0; +http://www.bing.com/bingbot.htm)'),

    "Slurp": UserAgentPolicy("Slurp",'Mozilla/5.0 (compatible; Yahoo! Slurp; http://help.yahoo.com/help/us/ysearch/slurp)'),

    "DuckDuckbot": UserAgentPolicy("DuckDuckbot",'DuckDuckBot/1.0; (+http://duckduckgo.com/duckduckbot.html)'),

    "Yandex": UserAgentPolicy("Yandex", 'Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)'),

    "*": UserAgentPolicy("*",'Mozilla/5.0 (compatible; MyBot/1.0; +http://example.com/bot)')
}


#Parser

In [5]:
# questo funziona
class Parser:
    def __init__(self):
        self.logger = logging.getLogger(__name__)

    def normalize_url(self, url):
        try:
            uri = rfc3986.uri_reference(url).normalize()
            return uri.unsplit()
        except Exception as e:
            self.logger.error("Error normalizing %s: %s", url, e)
            return url

    def check_spider_traps(self, url):
        MAX_URL_LENGTH = 200
        MAX_PATH_DEPTH = 6
        trap_pattern = re.compile(r"(calendar|sessionid|track|ref|sort|date=|page=\d{3,})", re.IGNORECASE)

        link = urlparse(url)

        if len(url) > MAX_URL_LENGTH:
            return False
        if link.path.count('/') > MAX_PATH_DEPTH:
            return False
        if trap_pattern.search(url):
            return False
        logging.info(f"{url} è sicuro")
        return True

    async def fetch_sitemap(self, session, sitemap_url):
        try:
            async with session.get(sitemap_url, timeout=10) as response:
                if response.status != 200:
                    self.logger.warning(f"Failed to fetch {sitemap_url} (Status {response.status})")
                    return []

                content_type = response.headers.get("Content-Type", "").lower()
                content = await response.text()

                # XML Sitemap
                if "xml" in content_type:
                    try:
                        root = ET.fromstring(content.encode())
                    except ET.ParseError as e:
                        self.logger.error(f"XML parse error in {sitemap_url}: {e}")
                        return []

                    namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

                    if root.tag.endswith('index'):
                        sub_sitemaps = [loc.text for loc in root.findall('ns:sitemap/ns:loc', namespace)]
                        results = []
                        for sub_url in sub_sitemaps:
                            results.extend(await self.fetch_sitemap(session, sub_url))
                        return results

                    elif root.tag.endswith('urlset'):
                        return [{
                            'url': url.findtext('ns:loc', default='', namespaces=namespace),
                            'priority': url.findtext('ns:priority', default=None, namespaces=namespace),
                            'update': url.findtext('ns:changefreq', default=None, namespaces=namespace)
                        } for url in root.findall('ns:url', namespace)]

                # HTML Sitemap
                elif "html" in content_type:
                    soup = BeautifulSoup(content, "html.parser")
                    return [{
                        'url': self.normalize_url(a['href']),
                        'priority': None,
                        'update': None
                    } for a in soup.find_all('a', href=True)]

                else:
                    self.logger.warning(f"Unsupported format: {sitemap_url} ({content_type})")
                    return []

        except aiohttp.ClientError as e:
            self.logger.error(f"Error fetching {sitemap_url}: {e}")
            return []

    async def parse_sitemap(self, sitemap_list):
        if not sitemap_list:
            self.logger.warning("No sitemap available.")
            return pd.DataFrame(columns=['url', 'priority', 'update'])

        all_entries = []
        async with aiohttp.ClientSession() as session:
            tasks = [self.fetch_sitemap(session, url) for url in sitemap_list if url]
            results = await asyncio.gather(*tasks)

            for entries in results:
                for entry in entries:
                    url = self.normalize_url(entry['url'])
                    if self.check_spider_traps(url):
                        entry['url'] = url
                        all_entries.append(entry)

        print(f"sitemap urls{all_entries}")
        return pd.DataFrame(all_entries, columns=['url', 'priority', 'update']).drop_duplicates()

    def extract_links(self, html, sitemaps_urls=None, useragent=None):
        soup = BeautifulSoup(html, 'html.parser')
        urls = []

        for tag in soup.find_all(href=True):
            urls.append(tag['href'])
        for tag in soup.find_all(src=True):
            urls.append(tag['src'])

        if useragent and useragent.path_disallow:
            urls = [url for url in urls if all(path not in url for path in useragent.path_disallow)]

        urls = [self.normalize_url(url) for url in urls]
        urls = list(set(urls))
        urls = [url for url in urls if self.check_spider_traps(url)]

        if sitemaps_urls is not None:
            def_urls = set(sitemaps_urls)
            def_urls.update(url for url in urls if url not in def_urls)
            return list(def_urls)
        else:
            return urls

    def parse_page_tags_all(self, html, tags_type=None):
        if tags_type is None:
            tags_type = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'div', 'span', 'a']

        soup = BeautifulSoup(html, "html.parser")

        for tag in soup(['script', 'style', 'footer', 'nav', 'noscript', 'header', 'form', 'aside']):
            tag.decompose()

        for comment in soup.find_all(string=lambda text: isinstance(text, comment)):
            comment.extract()

        tags = soup.find_all(tags_type)
        print(f"sono state trovate {len(tags)} parole")
        return [tag.get_text(separator=' ', strip=True) for tag in tags]


#Storage

In [ ]:

#   Check if the necessary NLTK resources have been downloaded. Otherwise, download them.
for resource in ["stopwords", "wordnet", "punkt", "averaged_perceptron_tagger", "omw-1.4"]:
    try:
        nltk.data.find(f"corpora/{resource}")
    except LookupError:
        nltk.download(resource, quiet=True)


#   Words to remove during preprocessing
NUMBER_WORDS = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']


def get_wordnet_pos(tag):
    """
    Maps POS tag to WordNet format.
    Input:
        tag (str): POS tag in Penn Treebank format.
    Output:
        WordNet POS tag constant (e.g., NOUN, VERB, ADJ, ADV).
    Description:
        Converts common Penn Treebank POS tags to the corresponding
        WordNet POS constants used for lemmatization.
    """
    if tag.startswith('J'): return ADJ
    if tag.startswith('V'): return VERB
    if tag.startswith('N'): return NOUN
    if tag.startswith('R'): return ADV
    return NOUN


def preprocess_sync(text):
    """
    Synchronously preprocesses text by tokenizing, filtering, POS tagging,
    and lemmatizing.
    Input:
        text (str): Raw input text.
    Output:
        list of str: Preprocessed and lemmatized tokens.
    Description:
        Performs tokenization, removes stopwords and numeric words, tags POS,
        and lemmatizes tokens to their base form.
    """
    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(text.lower())
    words = [w for w in words if w.isalpha() and w not in stop_words and w not in NUMBER_WORDS]
    tagged_words = pos_tag(words)
    return [lemmatizer.lemmatize(w, get_wordnet_pos(t)) for w, t in tagged_words]


async def preprocess(text):
    """
    Asynchronously preprocesses text by running synchronous preprocessing
    in a separate thread.
    Input:
        text (str): Raw input text.
    Output:
        list of str: Preprocessed and lemmatized tokens.
    Description:
        Wraps synchronous preprocessing to avoid blocking the async event loop.
    """
    return await asyncio.to_thread(preprocess_sync, text)


async def content_page(url):
    """
    Fetches HTML content from a URL asynchronously and parses text content.
    Input:
        url (str): URL of the webpage.
    Output:
        str: Concatenated textual content extracted from the page.
    Description:
        Uses asynchronous fetch and synchronous parser to retrieve and process
        page content for indexing or analysis.
    """
    html = await fetcher.fetch(url)
    text_from_page = parser.parse_page_tags_all(html)
    return ' '.join(text_from_page)


def compute_fingerprint(text):
    """
        Computes a 64-bit Simhash fingerprint for a given text.
    Input:
        text (str): Text to fingerprint.
    Output:
        int: Simhash fingerprint value.
    Description:
        Uses synchronous preprocessing to tokenize and lemmatize the text,
        then computes a Simhash to represent the text compactly for near-duplicate detection.
    """
    words = preprocess_sync(text)
    return simhash(words).value


def hamming_distance(fp1, fp2):
    """
    Computes the Hamming distance between two 64-bit integer fingerprints.
    Input:
        fp1 (int): First fingerprint.
        fp2 (int): Second fingerprint.
    Output:
        int: Number of differing bits.
    Description:
        Counts differing bits between two fingerprints to measure similarity.
    """
    x = (fp1 ^ fp2) & ((1 << 64) - 1)
    distance = 0
    while x:
        distance += 1
        x &= x - 1
    return distance


def compute_age(lambda_, t):
    """
    Computes the age of content using an exponential decay model.
    Input:
        lambda_ (float): Decay rate parameter.
        t (float): Time since last fetch (e.g., in days).
    Output:
        float: Computed age score.
    Description:
        Models content freshness with an aging function based on decay lambda.
    """
    if lambda_ == 0:
        return t
    return (t + lambda_ * np.exp(-lambda_ * t) - 1) / lambda_


def calculate_page_type(content, url=""):
    """
    Classifies a page type based on URL and content heuristics.
    Input:
        content (str): Text content of the page.
        url (str): URL of the page (optional).
    Output:
        str: One of "frequent", "occasional", "static", or "default".
    Description:
        Uses keywords and URL patterns to assign a frequency category to the page.
    """
    content = content.lower()
    url = url.lower()
    if "guardian" in url or "cnn.com" in url or "bbc.com" in url:
        if "live" in content and "update" in content:
            return "frequent"
    if "wikipedia.org/wiki/" in url:
        return "static"
    if "live" in url or "breaking" in url:
        return "frequent"
    if any(k in url for k in ["calendar", "event", "workshop", "conference"]):
        return "occasional"
    if any(k in url for k in ["about", "privacy", "contact", "terms"]):
        return "static"
    frequent_keywords = ["breaking news", "live updates", "as it happens", "developing story"]
    occasional_keywords = ["calendar", "workshop", "conference", "event", "seminar"]
    static_keywords = ["contact us", "about us", "privacy policy", "company info", "terms of service"]
    freq_count = sum(kw in content for kw in frequent_keywords)
    occas_count = sum(kw in content for kw in occasional_keywords)
    static_count = sum(kw in content for kw in static_keywords)
    if freq_count >= 2:
        return "frequent"
    if occas_count >= 1:
        return "occasional"
    if static_count >= 1:
        return "static"
    return "default"


def from_gap_encoding(gaps):
    """
    Decodes a list of gap-encoded positions back into absolute positions.
    Input:
        gaps (list of int): Gap-encoded positions.
    Output:
        list of int: Absolute positions.
    Description:
        Reconstructs absolute token positions from gap-encoded format.
    """
    if not gaps:
        return []
    positions = [gaps[0]]
    for gap in gaps[1:]:
        positions.append(positions[-1] + gap)
    return positions


def to_gap_encoding(positions):
    """
    Encodes a list of absolute positions into gap-encoded format.
    Input:
        positions (list of int): Absolute token positions.
    Output:
        list of int: Gap-encoded positions.
    Description:
        Compresses positions by storing gaps between successive positions.
    """
    if not positions:
        return []
    return [positions[0]] + [positions[i] - positions[i-1] for i in range(1, len(positions))]

In [ ]:



#   Lambda mapping for different page types
LAMBDA_BY_TYPE = {
    "frequent": 2.0,
    "occasional": 1.0,
    "static": 0.05,
    "default": 0.5
}



class Storage:

    """
    Asynchronous storage manager for pages metadata and inverted index.

    Attributes:
        pages_file (str): Path to the JSON file storing pages metadata.
        index_file (str): Path to the JSON file storing inverted index.
        pages (dict): In-memory dictionary of pages metadata.
        inverted_index (dict): In-memory inverted index mapping terms to postings.
        _lock (asyncio.Lock): Async lock to protect concurrent file access.

    Methods:
        __init__(pages_file, index_file):
            Synchronous constructor that sets file paths and initializes empty dictionaries for
            `pages` and `inverted_index`.
            It creates an asynchronous lock to manage concurrent write operations.

        async_init():
            Asynchronously loads data from JSON files.

        _load_json_async(filename):
            Loads JSON file asynchronously, returns dict or None.

        _save_json_async(filename, data):
            Saves data as JSON asynchronously under lock.

        save_page(url, content):
            Saves a page's metadata and indexes its terms asynchronously.

        index_terms(url, content, lock_acquired=False):
            Indexes terms of a page asynchronously, optionally assuming lock is held.

        _index_terms_internal(url, content):
            Internal method to perform term indexing and update inverted index.

        get_page(url):
            Retrieves metadata dictionary for a given URL.

        get_page_type(url):
            Retrieves the type classification of a stored page.

        get_last_fetch(url):
            Retrieves the last fetch timestamp of a stored page.

        get_tf(term):
            Retrieves term frequency postings for a term.

        needs_refresh(url):
            Determines if a page should be re-fetched based on its age and type.

        is_near_duplicate(content, threshold=5):
            Checks if the content is near-duplicate of any stored page.

        commit():
            Saves pages metadata and inverted index to disk asynchronously.
    """

    def __init__(self, pages_file="data/pages.json", index_file="data/inverted_index.json"):
        self.pages_file = pages_file
        self.index_file = index_file
        self.pages = {}
        self.inverted_index = {}
        self._lock = asyncio.Lock()

    async def async_init(self):
        self.pages = await self._load_json_async(self.pages_file) or {}
        self.inverted_index = await self._load_json_async(self.index_file) or {}
        logger.info(f"Storage initialized. Loaded {len(self.pages)} pages and {len(self.inverted_index)} terms.")

    async def _load_json_async(self, filename):
        if not os.path.exists(filename):
            return None
        try:
            async with aiofiles.open(filename, "r", encoding="utf-8") as f:
                content = await f.read()
            # Parse json in thread pool per sicurezza
            return await asyncio.to_thread(json.loads, content)
        except Exception as e:
            logger.error(f"Failed to load JSON file {filename}: {e}")
            return None

    async def _save_json_async(self, filename, data):
        if not data:
            logger.info(f"[DEBUG] {filename} vuoto. Scrittura saltata.")
            return
        async with self._lock:
            try:
                async with aiofiles.open(filename, "w", encoding="utf-8") as f:
                    await f.write(json.dumps(data, indent=2))
            except Exception as e:
                logger.error(f"Failed to save JSON file {filename}: {e}")

    def _load_json_sync(self, filename):
        if not os.path.exists(filename):
            return None
        try:
            with open(filename, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception as e:
            logger.error(f"Errore nel caricamento di {filename}: {e}")
            return None

    async def save_page(self, url, content):
        async with self._lock:
            now = time.time()
            page_type = calculate_page_type(content, url)
            fingerprint = compute_fingerprint(content)
            logger.info(f"Saving page: {url}")

            self.pages[url] = {
                "fingerprint": fingerprint,
                "page_type": page_type,
                "last_fetch": now
            }

            await self.index_terms(url, content, lock_acquired=True)

    async def index_terms(self, url, content, lock_acquired=False):
        if not lock_acquired:
            async with self._lock:
                await self._index_terms_internal(url, content)
        else:
            await self._index_terms_internal(url, content)


    async def _index_terms_internal(self, url, content):
        if url not in self.pages:
            raise ValueError(f"URL {url} not found in pages")

        words = await preprocess(content)
        tf = Counter()
        positions = {}

        for i, word in enumerate(words):
            tf[word] += 1
            positions.setdefault(word, []).append(i)

        for word in positions:
            positions[word] = to_gap_encoding(positions[word])

        # Remove old entries
        for term in list(self.inverted_index.keys()):
            if url in self.inverted_index[term]:
                del self.inverted_index[term][url]
                if not self.inverted_index[term]:
                    del self.inverted_index[term]

        for term in tf:
            if term not in self.inverted_index:
                self.inverted_index[term] = {}
            self.inverted_index[term][url] = {
                "tf": tf[term],
                "positions": positions[term]
            }

        # await self._save_json_async(self.index_file, self.inverted_index)

    def get_page(self, url):
        return self.pages.get(url)

    def get_page_type(self, url):
        page = self.pages.get(url)
        if page:
            return page.get("page_type")
        return None

    def get_last_fetch(self, url):
        page = self.pages.get(url)
        if page:
            return page.get("last_fetch")
        return None

    def get_tf(self, term):
        return self.inverted_index.get(term, {})

    def needs_refresh(self, url):
        page = self.pages.get(url)
        if not page:
            return True

        last_fetch = page.get("last_fetch")
        if last_fetch is None:
            return True

        page_type = page.get("page_type", "default")

        now = time.time()
        time_in_days = (now - last_fetch) / 86400
        lambda_ = LAMBDA_BY_TYPE.get(page_type, 0.5)
        threshold = 1 / lambda_

        age = compute_age(lambda_, time_in_days)
        return age > threshold

    def is_near_duplicate(self, content, threshold=5):
        new_fp = compute_fingerprint(content)
        for url, page_data in self.pages.items():
            fp = page_data.get("fingerprint")
            if fp is None:
                continue
            try:
                d = hamming_distance(new_fp, fp)
                if d <= threshold:
                    return True, url
            except Exception:
                continue
        return False, None

    async def commit(self):
        logger.info(f"[DEBUG] Tentativo di commit. pages: {len(self.pages)}, index: {len(self.inverted_index)}")
        if not self.pages and not self.inverted_index:
            logger.info("Nessun dato da salvare. Salvataggio saltato.")
            return

        async with self._lock:
            await asyncio.gather(
                self._save_json_async(self.pages_file, self.pages),
                self._save_json_async(self.index_file, self.inverted_index)
            )
        logger.info("Files pages.json and inverted_index.json saved!")


#Scheduler

In [6]:

class Scheduler:
    def __init__(self, max_concurrency, num_spiders):
        self.running = False
        self.frontier = asyncio.Queue() # URLs to crawl
        self.seen = set()   # tracks seen URLs
        self.visited = set()    # tracks visited URLs
        self.semaphore = asyncio.Semaphore(max_concurrency) # limits max parallel fetches
        self.host_locks = defaultdict(asyncio.Lock) #{}  # ensures one fetch per host at a time WILL REUSE SAME LOCK?
        self.retries = defaultdict(int) # dictionary keeps count of how many times retried each URL

        self.max_concurrency = max_concurrency
        self.num_spiders = num_spiders

        self.fetcher = Fetcher(None)
        self.parser = Parser()
        self.storage = Storage ()

    async def add_url(self, url):
        """
        Add a new URL to the frontier if it hasn't been seen.
        """
        if url not in self.seen:
            self.seen.add(url)
            await self.frontier.put(url)

    async def seed_urls(self, urls):
        """
        Seeds the initial URL, or list of URLs, into the queue.
        """
        for url in urls:
            await self.add_url(url)   # passing it to add_url for check

    def get_hostname(self, url):
        """
        Extract the hostname from a URL.
        """
        return urlparse(url).netloc

    async def get_url(self):
        """
        Get the next URL from the frontier.
        Wait asynchronously for a URL to be available in the queue.
        """
        return await self.frontier.get()

    def task_done(self):
        """
        Mark the current task as completed in the frontier.
        """
        self.frontier.task_done()

    async def handle_fetch_failure(self, url, exception):
        """
        Handles a failed fetch attempt: retries up to 2 times.
        """
        logger.error(f"Fetch failed for {url}: {exception}")
        self.retries[url] += 1

        if self.retries[url] <= 2:
            logger.info(f"Re-queuing {url} (attempt {self.retries[url]})")
            await self.frontier.put(url)
        else:
            logger.info(f"Giving up on {url} after {self.retries[url]} attempts.")

    async def fetch_url(self, url):
        """
        Fetches a URL with concurrency and politeness constraints.
        Handles retry on failure.
        """
        hostname = self.get_hostname(url)

        # enforce both global fetch concurrency and per-host politeness
        async with self.semaphore:
            async with self.host_locks[hostname]: ## get_host_lock(hostname)
                logger.info(f"Fetching {url}")
                start_time = time.perf_counter()    # want to measure time taken for each request
                try:
                    response = await self.fetcher.fetch(url)
                    duration = time.perf_counter() - start_time
                    logger.info(f"Fetched {url} in {duration:.2f}s")
                    return response
                except aiohttp.ClientError as e:
                    duration = time.perf_counter() - start_time
                    logger.error(f"Fetch failed for {url} after {duration:.2f}s: {e}")
                    await self.handle_fetch_failure(url, e)
                    return None
                except Exception as e:
                    duration = time.perf_counter() - start_time
                    logger.error(f"Fetch failed for {url} after {duration:.2f}s: {e}")
                    await self.handle_fetch_failure(url, e)
                    return None

    async def process_response(self, url, response):
        """
        Handles a successful fetch: parses, stores, and queues new URLs.
        """
        if not response:
            logger.warning(f"Empty response for {url}")
            return

        try:
            content, final_url, status = response
        except Exception as e:
            logger.error(f"Unexpected response format from {url}: {e}")
            return

        # Skip non-successful responses or empty content
        if status != 200 or not content:
            logger.info(f"Skipping {url} due to status {status} or empty content.")
            return

        try:
            await self.storage.save_page(final_url, content)
            logger.info(f"Saved {final_url}")
        except Exception as e:
            logger.error(f"Failed to save {final_url}: {e}")
            return

        # mark as successfully visited only after processing
        self.visited.add(final_url)

        try:
            # Extract and enqueue links found in the page
            links = self.parser.extract_links(content, final_url)
            for link in links:
                await self.add_url(link)
        except Exception as e:
            logger.error(f"Failed to parse links from {final_url}: {e}")

    async def spider(self):
        while self.running:
            try:
                url = await asyncio.wait_for(self.get_url(), timeout=5)
            except asyncio.TimeoutError:
                # Se nessuna URL arriva per 5 secondi, esci dal loop
                break

            response = await self.fetch_url(url)
            await self.process_response(url, response)
            self.task_done()

    async def run(self, seeds=None):
        if not seeds:
            seeds = ["https://example.com"]

        async with aiohttp.ClientSession() as session:
            self.fetcher = Fetcher(session)
            await self.seed_urls(seeds)

            unique_hosts = {urlparse(url).netloc for url in seeds}
            for host in unique_hosts:
                site_url = f"https://{host}"
                await self.fetcher.check_robots(site_url)

            self.running = True  # Attiva il flag di esecuzione
            spiders = [asyncio.create_task(self.spider()) for _ in range(self.num_spiders)]

            await self.frontier.join()  # Aspetta che la coda sia vuota
            self.running = False        # Ferma gli spider

            # Attendi che gli spider escano dal loop e terminino
            await asyncio.gather(*spiders, return_exceptions=True)

            logger.info("Crawling finished.")
            logger.info(f"Total seen URLs: {len(self.seen)}")
            logger.info(f"Total successfully visited URLs: {len(self.visited)}")



#Main

#Test 1#
questo funziona

In [ ]:
#questo funziona
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class Parser:
    def extract_links(self, content, base_url):
        # Per semplicità nessun link estratto
        return []

class Storage:
    async def save_page(self, url, content):
        logger.info(f"Saving page: {url} (content length {len(content)})")

class Scheduler:
    def __init__(self, max_concurrency, num_spiders):
        self.running = True
        self.frontier = asyncio.Queue()
        self.seen = set()
        self.visited = set()
        self.semaphore = asyncio.Semaphore(max_concurrency)
        self.host_locks = defaultdict(asyncio.Lock)
        self.retries = defaultdict(int)

        self.max_concurrency = max_concurrency
        self.num_spiders = num_spiders

        self.fetcher = Fetcher(None)
        self.parser = Parser()
        self.storage = Storage()

    async def add_url(self, url):
        if url not in self.seen:
            self.seen.add(url)
            await self.frontier.put(url)

    async def seed_urls(self, urls):
        for url in urls:
            await self.add_url(url)

    def get_hostname(self, url):
        return urlparse(url).netloc

    async def get_url(self):
        return await self.frontier.get()

    def task_done(self):
        self.frontier.task_done()

    async def fetch_url(self, url):
        hostname = self.get_hostname(url)

        async with self.semaphore:
            async with self.host_locks[hostname]:
                response = await self.fetcher.fetch(url)
                return response

    async def process_response(self, url, response):
        if not response:
            logger.warning(f"Empty response for {url}")
            return

        try:
            content, final_url, status = response
        except Exception as e:
            logger.error(f"Unexpected response format from {url}: {e}")
            print(f"Unexpected response format from {url}: {e}")
            return

        if status != 200 or not content:
            logger.info(f"Skipping {url} due to status {status} or empty content.")
            print(f"Skipping {url} due to status {status} or empty content.")
            return

        await self.storage.save_page(final_url, content)
        self.visited.add(final_url)

        try:
            links = self.parser.extract_links(content, final_url)
            for link in links:
                await self.add_url(link)
        except Exception as e:
            logger.error(f"Failed to parse links from {final_url}: {e}")

    async def spider(self):
        while self.running:
            try:
                url = await asyncio.wait_for(self.get_url(), timeout=2)
            except asyncio.TimeoutError:
                break
            response = await self.fetch_url(url)
            await self.process_response(url, response)
            self.task_done()

    async def run(self, seeds=None):
        if not seeds:
            seeds = ["https://example.com"]

        async with aiohttp.ClientSession() as session:
            self.fetcher = Fetcher(session)
            await self.seed_urls(seeds)

            # ✅ CHECK robots.txt UNA VOLTA per ogni dominio nei seed
            unique_hosts = {urlparse(url).netloc for url in seeds}
            for host in unique_hosts:
                site_url = f"https://{host}"
                await self.fetcher.check_robots(site_url)

            self.running = True  # Attiva il flag di esecuzione
            spiders = [asyncio.create_task(self.spider()) for _ in range(self.num_spiders)]

            await self.frontier.join()  # Aspetta che la coda sia vuota
            self.running = False        # Ferma gli spider

            # Attendi che gli spider escano dal loop e terminino
            await asyncio.gather(*spiders, return_exceptions=True)

            logger.info("Crawling finished.")
            print("Crawling finished.")
            logger.info(f"Total seen URLs: {len(self.seen)}")
            print(f"Total seen URLs: {len(self.seen)}")
            logger.info(f"Total successfully visited URLs: {len(self.visited)}")
            print(f"Total successfully visited URLs: {len(self.visited)}")

async def main():
    seeds = [
        "https://www.bbc.com/",

    ]
    scheduler = Scheduler(max_concurrency=3, num_spiders=2)
    await scheduler.run(seeds=seeds)

In [ ]:
await main()

#Test 2
ottengo tutti gli url trovati con lo scarping + parsing di una pagina

In [7]:

logging.basicConfig(level=logging.INFO)

class CustomScheduler(Scheduler):
    def __init__(self, max_concurrency, num_spiders):
        super().__init__(max_concurrency, num_spiders)
        self.extracted_urls = set()
        self.page_contents = {}  # {url: html}

    async def process_response(self, url, response):
        if not response:
            return

        content, final_url, status = response
        await self.storage.save_page(final_url, content)
        self.visited.add(final_url)


        self.page_contents[final_url] = content

        links = self.parser.extract_links(content)
        for link in links:
            self.extracted_urls.add(link)
            await self.add_url(link)



class Storage:
    async def save_page(self, url, content):
        logger.info(f"Saving page: {url} (content length {len(content)})")

async def main():
    # Setup
    seed_urls = ["https://www.bbc.com/",]  # Puoi cambiare
    parser = Parser()
    scheduler = CustomScheduler(max_concurrency=2, num_spiders=1)

    scheduler.parser = parser
    scheduler.fetcher = Fetcher(None)  # usa la tua fetcher senza sessione aiohttp qui
    scheduler.storage = Storage()

    # Avvia lo scraping
    await scheduler.seed_urls(seed_urls)
    await scheduler.run(seed_urls)

    # ✅ Stampa i risultati
    print("\n🔗 URL trovati:")
    for url in sorted(scheduler.extracted_urls):
        print(url)

    # ✅ Analizza una delle pagine trovate
    if scheduler.page_contents:
        sample_url, html = next(iter(scheduler.page_contents.items()))
        parsed_text = parser.parse_page_tags_all(html)

        print(f"\n🧠 Contenuto analizzato da: {sample_url}\n")
        for line in parsed_text[:10]:  # stampa solo i primi 10 pezzi
            print("-", line)

    else:
        print("Nessuna pagina disponibile per l'analisi.")


In [ ]:
await main()